# 5.1. download `production` artifacts

In [1]:
import os
import pandas as pd
from joblib import load

import wandb
from ex_wandb.utils.wandb_utils import get_wandb_artifact

from ex_wandb.ml_src.data import preprocess
from ex_wandb.ml_src.model import inference

In [2]:
run = wandb.init(project='ex_census_wandb', job_type='inference testing')

wandb: Currently logged in as: wg_lucas. Use `wandb login --relogin` to force relogin


In [3]:
model_artifact, model_path = get_wandb_artifact(run,
                               artifact_name="model", 
                               file_name="model.joblib", 
                               tag="prod", 
                               artifact_type='model_artifact')

encoder_artifact, encoder_path = get_wandb_artifact(run,
                               artifact_name="feature_engineering", 
                               file_name="encoder.joblib", 
                               tag="prod", 
                               artifact_type='feature_engineering_artifact')

lb_artifact, lb_path = get_wandb_artifact(run,
                               artifact_name="feature_engineering", 
                               file_name="lb.joblib", 
                               tag="prod", 
                               artifact_type='feature_engineering_artifact')

cat_features = encoder_artifact.metadata['categorical_feature']
label = encoder_artifact.metadata['label']

wandb: Downloading large artifact model:prod, 68.62MB. 1 files... Done. 0:0:0.0


In [4]:
run.finish()

# 5.2. execute inference on new data

In [5]:
model = load(model_path)
encoder = load(encoder_path)
lb = load(lb_path)

In [6]:
new_data = {"workclass": "Private",
            "education": "Doctorate",
            "marital-status": "Married-civ-spouse",
            "occupation": "Prof-specialty",
            "relationship": "Husband",
            "race": "White",
            "sex": "Male",
            "native-country": "United-States",
            "hours-per-week": 60}

new_data = pd.DataFrame.from_dict(new_data, orient='index').T

In [7]:
X_test, _, _, _ = preprocess(X=new_data, 
                             categorical_features=cat_features, 
                             training=False, 
                             encoder=encoder, 
                             lb=lb)

In [8]:
preds = inference(model, X_test)

In [9]:
output = lb.inverse_transform(preds)[0]
print(output)

>50K
